# 中国語学習データベースをNeo4j(グラフデータベース)で構築してみる

以前、中国語学習を熱心にやっていたときに、単語や単文のデータベースを作っていたが、あまり長続きしなかった。理由は、正規化の部分での時間的コストが掛かること。項目を増やしたり、テーブルを追加したりなどなどの作業に追われて、あまり効率的ではなかったのが原因ではないかと考えている。そこで、グラフデータベースを使って面白いことが出来ないかを試してみることにした。

やりたいことを列挙すると、こんな感じ。
- 短文を登録すると、こんな構文を使っているねとか、この単語を使っているねというのが、ある程度自動でリンクされる。
- 特定に構文とか言い回しとかに沿った単文を例文として一覧で列挙する。
- 読み方などが分からない単語を登録する。
などなど。

## 構造を考えてみる

Neo4jには、各ノードをLabelで性格付けできるので、こんな感じのモノを作ってみるイメージ。
- 単語(Word)
- カテゴリ(Category)品詞とか
- 構文(Structure) 把構文みたいな
- 単文(Sentence)

## テキスト/CSVファイルからの取り込み

まずは単語の入ったcsvファイルを読み込む。項目数が多いので、DictReaderで辞書として取り込み、それをリスト化する

In [2]:
import csv
word_list = []
with open('chinesewords_20160219.csv', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        word_list.append(row)
print('{} words stored in word_list'.format(len(word_list)))

5809 words stored in word_list


同じく単文の入ったtxtファイルを読み込む。こちらは項目数が今のところ1つだけなので、csvとして読む必要はない。ただし、行末の改行(\n)を消してやる必要あり。

In [3]:
sentence_list = []
with open('chinese_sentences20160218.txt', encoding='utf-8') as textfile:
    sentence_list = [row.rstrip() for row in textfile]
print('{} sentences stored in sentenc_list'.format(len(sentence_list)))

569 sentences stored in sentenc_list


改行が消えているかの確認。

In [4]:
sentence_list[-1]

'那么大的鱼，根本钓不到。'

## Neo4jとの接続

Neo4jとつないでみよう。あらかじめpy2neoをインストールしておく。
```
$pip install py2neo
```

In [9]:
from py2neo import Graph, authenticate
from py2neo import Node, Relationship
authenticate('localhost:7474', 'neo4j', 'admin')
graph = Graph()    #http://localhost:7474

(リカバリ用)すべてのノードと関係を削除する場合は、これを使う。

In [25]:
graph.delete_all()

CategoryとSentenceについては、単一の属性でユニークになるのでuniqueness_constraintを作っておく。

In [10]:
graph.schema.create_uniqueness_constraint("Category", "category")
graph.schema.create_uniqueness_constraint("Sentence", "sentence")

Wordについては、多音字を想定してkeywordとpinyinの組み合わせでユニークとする。複数項目の指定は出来ないので、エントリ追加時のロジックで対応する。

まずは短文登録を行う。重複ケースを考えてmerge_oneを使う。uniqueness_constraintとセットで使うとドキュメントに書いてある。

In [11]:
for sentence in sentence_list:
    graph.merge_one("Sentence", "sentence", sentence)

続いて単語の登録。ロジックを工夫しながらやる必要あり。

In [12]:
counter_a = 0
counter_c = 0
for word in word_list:
    counter_a += 1
    ws = list(graph.find("Word", "keyword", word['word']))
    found = False
    for w in ws:
        if w["pinyin"] == word['pinyin']:
            found = True
            break
    if found == True:
        new_word = w
    else:
        counter_c += 1
        new_word = Node("Word", keyword=word['word'], pinyin=word['pinyin'], pinyinf=word['pinyinf'], meaning=word['meaning'])
        graph.create(new_word)
    found = False
    if word.get('category'):
        cs = list(graph.find("Category", "category", word['category']))
        if len(cs) == 0:
            new_category = Node("Category", category=word['category'])
            graph.create(new_category)
        else:
            new_category = cs[0]
        word_is_categorized_as_category = Relationship(new_word, 'is_categorized_as', new_category)
        graph.create(word_is_categorized_as_category)
    
print("all count:{0}, created words:{1}".format(counter_a, counter_c))    

all count:5809, created words:5806


少し時間が掛かりすぎか？

## データベース内のデータのハンドリング(検索)

既存のデータの検索について、方法を考える。まずは、cypherを直接実行するexecuteを使うパターン。

In [14]:
w = graph.cypher.execute("MATCH (w:Word) WHERE w.keyword='熬夜' RETURN w") 

In [15]:
print(w)

   | w                                                                         
---+----------------------------------------------------------------------------
 1 | (n571:Word {keyword:"熬夜",meaning:"徹夜する",pinyin:"ao2 ye4",pinyinf:"áo yè"})



In [16]:
type(w)

py2neo.cypher.core.RecordList

In [17]:
w[0]

 w                                                                         
----------------------------------------------------------------------------
 (n571:Word {keyword:"熬夜",meaning:"徹夜する",pinyin:"ao2 ye4",pinyinf:"áo yè"})

In [21]:
type(w[0])

py2neo.cypher.core.Record

In [ ]:
速さは問題ないが、取り回しがよく分からない。続いては、findを使う方法。

In [22]:
w = list(graph.find("Word", "keyword", "熬夜"))

In [23]:
w

[<Node graph='http://localhost:7474/db/data/' ref='node/571' labels={'Word'} properties={'pinyin': 'ao2 ye4', 'pinyinf': 'áo yè', 'meaning': '徹夜する', 'keyword': '熬夜'}>]

In [24]:
w[0]["keyword"]

'熬夜'

これの方が取り回しは簡単な気がする。とりあえず汎用的に使えるように関数を書いてみる。

In [29]:
def search_word(keyword):
    ws = list(graph.find("Word", "keyword", keyword))
    if len(ws) == 0:
        print("Keyword {0} not found in database".format(keyword))
        return None
    else:
        print("Found {0} entries for keyword {1} in database".format(len(ws), keyword))
        return ws    

In [30]:
word = search_word("菩萨")
print(word)

Found 1 entries for keyword 菩萨 in database
[<Node graph='http://localhost:7474/db/data/' ref='node/1199' labels={'Word'} properties={'pinyin': 'pu2 sa4', 'pinyinf': 'pú sà', 'meaning': '菩薩', 'keyword': '菩萨'}>]


続いてピンイン検索も。pinyinfでの検索も考えた方が良いかもしれないが、仕組みは同じで簡単に作れるので、とりあえずpinyinだけで作る。

In [32]:
def search_pinyin(pinyin):
    ws = list(graph.find("Word", "pinyin", pinyin))
    if len(ws) == 0:
        print("No keywords found to pronounce like {0}".format(pinyin))
        return None
    else:
        print("Found {0} entries to pronounce like {1}".format(len(ws), pinyin))
        return ws

In [41]:
words = search_pinyin('cheng2')
print(words)

Found 5 entries to pronounce like cheng2
[<Node graph='http://localhost:7474/db/data/' ref='node/2101' labels={'Word'} properties={'pinyin': 'cheng2', 'pinyinf': 'chéng', 'meaning': '盛る', 'keyword': '盛'}>, <Node graph='http://localhost:7474/db/data/' ref='node/2102' labels={'Word'} properties={'pinyin': 'cheng2', 'pinyinf': 'chéng', 'meaning': '', 'keyword': '橙'}>, <Node graph='http://localhost:7474/db/data/' ref='node/2103' labels={'Word'} properties={'pinyin': 'cheng2', 'pinyinf': 'chéng', 'meaning': '', 'keyword': '乘'}>, <Node graph='http://localhost:7474/db/data/' ref='node/6200' labels={'Word'} properties={'pinyin': 'cheng2', 'pinyinf': 'chéng', 'meaning': '', 'keyword': '城'}>, <Node graph='http://localhost:7474/db/data/' ref='node/6305' labels={'Word'} properties={'pinyin': 'cheng2', 'pinyinf': 'chéng', 'meaning': '', 'keyword': '成'}>]


ひとまず大丈夫そう。

In [42]:
len(word_list)

5809

In [43]:
len(sentence_list)

569